In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import folium

def plot_lat_lon_points(real_csv_path, generated_csv_path, output_file=None):
    """
    Reads two CSV files in the format:
        label, tid, lat, lon, day, hour, category
    and generates a scatter plot of latitude vs longitude
    to compare real data points and generated data points.

    Displays min/max lat/lon in the console and in the chart.
    """
    # Read the real data
    df_real = pd.read_csv(real_csv_path)
    # Read the generated data
    df_generated = pd.read_csv(generated_csv_path)

    # Combine them temporarily to compute overall min/max
    df_combined = pd.concat([df_real, df_generated], ignore_index=True)

    lat_min = df_combined['lat'].min()
    lat_max = df_combined['lat'].max()
    lon_min = df_combined['lon'].min()
    lon_max = df_combined['lon'].max()

    # Print the results in the console
    print(f"Latitude range: {lat_min} to {lat_max}")
    print(f"Longitude range: {lon_min} to {lon_max}")

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot real data points
    ax.scatter(
        df_real['lon'], df_real['lat'], 
        s=40, c='blue', alpha=0.6, label='Real'
    )

    # Plot generated data points
    ax.scatter(
        df_generated['lon'], df_generated['lat'], 
        s=40, c='red', alpha=0.6, label='Generated'
    )

    # Label axes and title
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Real vs Generated Data Points')

    # Display min/max lat/lon on the plot
    ax.text(
        0.05, 0.95,
        f"Lat range: {lat_min:.4f} - {lat_max:.4f}\n"
        f"Lon range: {lon_min:.4f} - {lon_max:.4f}",
        transform=ax.transAxes,
        va='top',
        ha='left',
        fontsize=9,
        color='black',
        bbox=dict(boxstyle='round,pad=0.4', fc='white', ec='black', alpha=0.7)
    )

    # Legend
    ax.legend()

    # Optionally save the figure
    if output_file:
        plt.savefig(output_file, dpi=300, bbox_inches='tight')

    # Show the plot
    plt.show()

    return lat_min, lat_max, lon_min, lon_max


def show_nyc_region_map(lat_min, lat_max, lon_min, lon_max, output_html='nyc_region_map.html'):
    """
    Displays a Folium map spanning the bounding box (lat_min, lon_min) to (lat_max, lon_max).
    Draws a rectangle for that bounding area.
    """
    # Determine the center (midpoint of bounding box)
    center_lat = (lat_min + lat_max) / 2
    center_lon = (lon_min + lon_max) / 2
    
    # Create a folium map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=9)

    # Define bounding coordinates: [SW corner, NE corner]
    bounds = [(lat_min, lon_min), (lat_max, lon_max)]
    
    # Fit the map to these bounds
    m.fit_bounds(bounds)
    
    # Draw a rectangle to visualize the bounding box
    folium.Rectangle(
        bounds=bounds,
        color='blue',
        fill=False
    ).add_to(m)
    
    # Save to HTML
    m.save(output_html)
    print(f"Map saved to {output_html}. Open it in a browser to view.")


if __name__ == "__main__":
    # Example usage
    lat_min, lat_max, lon_min, lon_max = plot_lat_lon_points(
        real_csv_path="syn_traj_test.csv",
        generated_csv_path="syn_traj_test (6).csv",
        output_file="comparison_plot.png"
    )

    show_nyc_region_map(lat_min, lat_max, lon_min, lon_max, output_html='nyc_region_map.html')
